# How tos

Here we list a few common use cases and how you can use this package to address these use cases.

## How to install `aermanager`?

Open a terminal and navigate to your favorite software repository location.

```
$ pip install aermanager
```

Then in a python terminal you should be able to import aermanager

In [1]:
import aermanager

Change directory in case you are running this notebook from the original location in the repository

In [2]:
cd ../../..

/home/sadique/git/aermanager


## How to load events from an *.aedat4 file?

In [3]:
from aermanager.aerparser import load_events_from_file
from aermanager.parsers import parse_aedat4

filename = "test/data/class2/data_sample.aedat4"
shape, events = load_events_from_file(filename, parser=parse_aedat4)
print(f"The shape of the sensor recording the data was inferred to be {shape}")

The shape of the sensor recording the data was inferred to be (240, 320)


In [4]:
events

array([(152, 126, 1605116853975736,  True),
       (148,  79, 1605116853975738, False),
       (146,  77, 1605116853975740, False), ...,
       ( 74, 227, 1605116855435678,  True),
       ( 42, 157, 1605116855435698, False),
       ( 41, 216, 1605116855435698, False)],
      dtype=[('x', '<u2'), ('y', '<u2'), ('t', '<u8'), ('p', '?')])

## How to load events from an *.aedat file?

You will still use the `load_events_from_file` method but you will need to specify the `parser`. You can use the following inbuilt parsers for the corresponding sensors: 
1. 'DVS128': use `parse_dvs128` (128, 128)
2. 'mini346': use `parse_346mini` (260, 346)
3. 'redV3': use `parse_red` (132,104)

## How to slice events ?

An event list extracted from a file could contain a very large sequence of events. Typically such a long spike list is not directly usable. 

For instance, if we are trying to train a network on such data, we typically want to present a network a series of images, where each image corresponds to a small slice of events. These smaller slices are generated typically using one of two criterion:

###  1. Slice by spike count

Here we define a slice as comprising of a fixed number of events (irrespective of the time length of each such slice). This strategy enables a way of auto-normalizing the data.

In [5]:
import numpy as np
from aermanager.preprocess import slice_by_count

# Generate indices to split the data every for 3000 spikes
indices = slice_by_count(events, spike_count=3000)

# Slice the data
sliced_events = np.split(events, indices)[:-1]

## Number of slices
len(sliced_events)

107

In [6]:
sliced_events[0].shape, sliced_events[0].dtype

((3000,), dtype([('x', '<u2'), ('y', '<u2'), ('t', '<u8'), ('p', '?')]))

### 2. Slice by time window

Here we define a slice as comprising of all events within a given time-window. This strategy ensures that the data has fixed time bounds and could be used to avoid blurring effects when creating frames or for learning spatio-temporal features when including time during training.

In [7]:
from aermanager.preprocess import slice_by_time

# Generate indices to split the data at every 3ms (3000 us)
indices = slice_by_time(events, time_window=3000)

# Slice the data
sliced_events = np.split(events, indices)[:-1]

## Number of slices
len(sliced_events)

486

## How to generate frames from events ?

In [8]:
from aermanager.preprocess import accumulate_frames

frames = accumulate_frames(
    sliced_events, 
    bins_y=range(shape[0]+1), 
    bins_x=range(shape[1]+1))
frames.shape

(486, 2, 240, 320)

## How to generate spike-rasters from events ?

In [9]:
from aermanager.preprocess import create_raster_from_xytp


# Let us consider the first slice we created above
events_xytp = sliced_events[0]

# dt is the time step we will use to rasterize.
dt = 1000  #ms

# Convert this set of events to a raster
spike_raster = create_raster_from_xytp(
    events_xytp, 
    dt=dt, 
    bins_y=range(shape[0]+1), 
    bins_x=range(shape[1]+1))

# The spike raster should be of shape [time, polarity, height, width]
print(f"Spike raster has the shape: {spike_raster.shape}")

# The total number of spikes in this slice should be equal to the sum of all the values in the spike_raster
print(f"The number of events in the raster {spike_raster.sum()} should match total events {len(events_xytp)}")

Spike raster has the shape: (3, 2, 240, 320)
The number of events in the raster 674.0 should match total events 674


The spike raster contains number of spikes produced in a given time window by any given neuron of a given polarity. We see that the slice we generated above had a lenth of 3 time steps of $1ms$ each. 

This format of data is useful as data input to `sinabs` models.

## How to crop events ?

In [10]:
from aermanager.preprocess import crop_events

left, right = 35,35
top, bottom = 4, 4

events_cropped, new_shape = crop_events(
    events, 
    input_shape=shape, 
    crop_size=((left, right), (top, bottom)))

new_shape

(232, 250)

## How to generate frames and slices directly from a .aedat4 file ?

We have some convenience functions to directly load aedat4 files and preprocess into slices and corresponding frames.

NOTE: This only works for aedat4 files where the shape of the data is inferred directly from file.

In [11]:
from aermanager.dataset_generator import dataset_content_generator

sliced_events, frames, _ = dataset_content_generator(filename, time_window=3000)

frames.shape

(486, 2, 240, 320)

The above method can be used to slice data either by `time_window` or by `spike_count` by passing the corresponding named parameter to the method. You can also crop the data by passing a `crop_size` parameter. 

## How to generate a dataset from a list of files and labels ?

For a typical machine learning task, we will need to use multiple data recordings and correspoindin file labels structured as a dataset. Such a dataset could then be used to train and test models.

You can generate such a dataset using the method `gen_dataset_from_list`.

In [12]:
from aermanager.dataset_generator import gen_dataset_from_list

# Suppress all warnings
import warnings
warnings.filterwarnings('ignore')


event_files = [
    "test/data/class1/test.aedat4", 
    "test/data/class2/data_sample.aedat4"
]

labels = ["class1", "class2"]


gen_dataset_from_list(
    event_files=event_files, 
    labels=labels, 
    destination_path="./example_dataset/",
    time_window=30000,
    parser=parse_aedat4,
)

2it [00:00,  4.89it/s]


This will create a set of files and folders, each corresponding to one data recording.

In [13]:
ls -R "./example_dataset/"

./example_dataset/:
data_sample_bcac0/  test_a98d9/

./example_dataset/data_sample_bcac0:
0.h5   14.h5  19.h5  23.h5  28.h5  32.h5  37.h5  41.h5  46.h5  7.h5
10.h5  15.h5  1.h5   24.h5  29.h5  33.h5  38.h5  42.h5  47.h5  8.h5
11.h5  16.h5  20.h5  25.h5  2.h5   34.h5  39.h5  43.h5  4.h5   9.h5
12.h5  17.h5  21.h5  26.h5  30.h5  35.h5  3.h5   44.h5  5.h5
13.h5  18.h5  22.h5  27.h5  31.h5  36.h5  40.h5  45.h5  6.h5

./example_dataset/test_a98d9:
0.h5   11.h5  13.h5  15.h5  17.h5  1.h5  3.h5  5.h5  7.h5  9.h5
10.h5  12.h5  14.h5  16.h5  18.h5  2.h5  4.h5  6.h5  8.h5


### CSV files
For convenience, the files and labels can also be passed via a csv file by using the method `gen_dataset_from_csv`.

### Folders
Alternatively, if your data is stored in structured folders, with each folder corresponding to a label, you can use the method `gen_dataset_from_folders`.

## How to load a dataset (for a dataloader)

Once you have extracted and created a set of files for a dataset as described above, you can use our convenience dataloaders as follows.

In [14]:
from aermanager.datasets import SpikeTrainDataset, FramesDataset

# A dataset with sliced spike trains
st_dataset = SpikeTrainDataset(source_folder="./example_dataset/")  

# A dataset for pre-generated frames
frames_dataset = FramesDataset(source_folder="./example_dataset/")


You can iterate over the data from the dataset objects like any regular iterator, or pass it to your favorite dataloader. In the below cell we print all the labels in the dataset we just used.

In [15]:
[label for data, label in frames_dataset][:5]

[b'class2', b'class2', b'class2', b'class2', b'class2']